In [52]:
import os
import geopandas as gpd
import json
from datetime import datetime, timedelta
import posixpath as path
import pandas as pd




In [59]:
# Define parameters
region = 'central'  # options: 'north', 'central', 'mediterranean'
ens = 'ssp2'              # options: 'historical', 'era5-land', 'ssp1', 'ssp3', 'ssp2'
years = '2021-2040'       # options: '1991-2010', '2021-2040', '2041-2060', '2061-2080', '2081-2100'

# Build file path
results_dir = path.join('/home/jsoma/results', region, f'{ens}_{years}')
print(results_dir)
name = f'Simulated_Burned_Area_{ens}_{years}.shp'
file_path = path.join(results_dir, f'Simulated_Burned_Area_{ens}_{years}.shp')

# Load file
gdf = gpd.read_file(file_path)

# Fix invalid geometries
gdf['geometry'] = gdf['geometry'].buffer(0)

# Set CRS to EPSG:3035 if not already set
if gdf.crs is None or gdf.crs.to_epsg() != 3035:
    gdf = gdf.set_crs('EPSG:3035', allow_override=True)

# Calculate area in hectares
gdf['area_ha'] = gdf.geometry.area / 10000  # 1 ha = 10,000 m²

if years == '2021-2040':

    # Convert 'date' from string to datetime
    gdf['date'] = pd.to_datetime(gdf['date'])

    # Add 30 years to the date
    gdf['date'] = gdf['date'] + pd.DateOffset(years=30)
    gdf['date'] = gdf['date'].dt.strftime('%Y-%m-%d')

# save teh file 
save_dir =  '/home/jsoma/IIASA-Resuls'

ens_save = ens + '_' + years
if region == 'mediterranean':
    region = 'South'
    save_dir = os.path.join(save_dir, 'South',ens_save)
elif region == 'central':
    region = 'Central'
    save_dir = os.path.join(save_dir, region, ens_save)
elif region == 'north':
    region = 'North'
    save_dir = os.path.join(save_dir, region, ens_save)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_name =f"ForeFireSim_burned_area_{ens}_{years}_{region}.shp"
save_path = os.path.join(save_dir, save_name)
gdf.to_file(save_path, driver='ESRI Shapefile')



/home/jsoma/results/central/ssp2_2021-2040
